# Test 2b - Merge, Aggregate, Visualize

## Introduction

This problem tests your ability to merge, aggregate, and visualize data.
You'll work with realistic business data to answer questions about product profitability.

Before submitting, ensure your notebook:

- Includes your verification info (Banner ID and code) as the first cell
- Runs completely with "Kernel → Restart & Run All"
- Shows your work (don't just print final answers)
- Includes your interpretation responses at the end

## Dataset: Bean & Brew Coffee Shop

You've been hired to analyze sales data for Bean & Brew, a local coffee shop chain with three locations. The owner wants to understand which products are most profitable and how profitability varies across locations.

There are two data files:

- `sales.csv` - Individual transaction records
  - `transaction_id`: Unique transaction identifier
  - `timestamp`: Date and time of purchase (format: 'YYYY-MM-DD HH:MM:SS')
  - `location`: Store location ('Downtown', 'Campus', 'Suburb')
  - `product_id`: Product identifier (links to products table)
  - `quantity`: Number of units sold
  - `revenue`: Total revenue from transaction
  - `payment_method`: Payment type ('Cash', 'Credit', 'Mobile')
  - `loyalty_member`: Boolean indicating if customer is loyalty program member
- `products.csv` - Product catalog
  - `product_id`: Unique product identifier
  - `name`: Product name
  - `category`: Product category ('Coffee', 'Tea', 'Pastry', 'Sandwich')
  - `cost`: Cost to make/acquire one unit (for profit calculation)

The dataset contains two weeks of sales data (approximately 3,000 transactions) across 18 products. It includes both regular and premium items in each category.

We are interested in learning which product categories generate the most profit, and how does profitability vary across locations? Complete the following tasks to answer those questions and more.

## Scoring

Total: 30 points (+ 5 bonus)

- Tasks 1-6: 18 points
- Interpretation (Questions 1-4): 12 points
- Bonus: 5 points (optional)

## Generate Your Dataset

Before starting the analysis, you need to generate your unique dataset using your Banner ID.

1. Open a terminal in the folder containing `generate_test2b_data.py`
2. Activate your conda virtual environment
3. Run: `python generate_test2b_data.py`
4. Enter your 9-digit Banner ID when prompted (you'll enter it twice to confirm)

The script will create two files in the `data/` folder:

- `data/products.csv` - Product catalog
- `data/sales.csv` - Transaction records

Important: At the end, the script displays a verification block like this:

```
======================================================================
DATA VERIFICATION INFORMATION
======================================================================
banner id:          904123456
verification code:  5e0dXXXXXXXX
======================================================================
```

Copy the banner id and verification code lines and paste them into the cell below. This is required for submission.

```
======================================================================
DATA VERIFICATION INFORMATION
======================================================================
banner id:          904430761
verification code:  03bc434b7add
======================================================================

```

## Setup: Load the Data

Run the cell below to load the data. The code is provided for you.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data (provided)
sales = pd.read_csv('data/sales.csv', parse_dates=['timestamp'])
products = pd.read_csv('data/products.csv')

print(f"Sales: {len(sales)} transactions")
print(f"Products: {len(products)} items in catalog")
print()
print("Sales preview:")
sales.head()

Sales: 2581 transactions
Products: 18 items in catalog

Sales preview:


,transaction_id,timestamp,location,product_id,quantity,revenue,payment_method,loyalty_member
0,1003,2025-11-13 06:00:00,Campus,108,1.0,2.61,Credit,False
1,1001,2025-11-13 06:13:00,Campus,104,1.0,2.17,Mobile,False
2,1000,2025-11-13 06:44:00,Downtown,103,1.0,5.89,Credit,False
3,1002,2025-11-13 06:49:00,Downtown,101,2.0,6.48,Credit,False
4,1016,2025-11-13 07:03:00,Campus,102,1.0,4.43,Credit,False


In [3]:
# Products catalog
products

,product_id,name,category,cost
0,101,Espresso,Coffee,1.2
1,102,Cappuccino,Coffee,1.8
2,103,Latte,Coffee,1.9
3,104,Americano,Coffee,1.0
4,105,Cold Brew,Coffee,1.5
5,106,Nitro Cold Brew,Coffee,2.5
6,107,Green Tea,Tea,0.8
7,108,Chai Latte,Tea,1.2
8,109,Herbal Tea,Tea,0.7
9,110,Premium Matcha,Tea,2.2


## Task 1: Data Quality Check

Investigate the sales data quality and coverage. Your code should answer the following questions:

1. How many missing values are in each column?
2. How many unique product IDs appear in the sales data? Compare this to the catalog size.
3. What is the date range (earliest and latest transaction)?

### Your Code

In [5]:
# Task 1: use isna(), nunique(), min/max on timestamp

# 1
sales["timestamp"] = pd.to_datetime(sales["timestamp"])
missing_values = sales.isna().sum()

print(f"Missing values per column:\n{missing_values}")

Missing values per column:
transaction_id    0
timestamp         0
location          0
product_id        0
quantity          3
revenue           5
payment_method    0
loyalty_member    0
dtype: int64


In [11]:
# 2
unq_sales_products = sales["product_id"].nunique()
catalog_size = len(products)

print(f"Unique product IDs in sales: {unq_sales_products}")
print(f"Size of the catalog: {catalog_size}")

Unique product IDs in sales: 21
Size of the catalog: 18


In [20]:
earliest_transac = sales["timestamp"].min()
latest_transac = sales["timestamp"].max()

print(f"The date range is from {earliest_transac} to {latest_transac}")

The date range is from 2025-11-13 06:00:00 to 2025-11-26 21:37:00


## Task 2: Merge Sales with Products

Merge the sales and products DataFrames to add product information (name, category, cost) to each transaction.

### Your Code

In [22]:
# Task 2: use pd.merge() with appropriate join type
sales_w_prod_info = pd.merge(sales, products, on="product_id", how="left")

## Task 3: Verify the Merge

After merging, verify that the merge worked correctly:

1. Compare the shape of the merged DataFrame to the original sales data
2. Check if any rows have missing values in the product columns (name, category, cost)
3. If there are missing values, identify which product_id(s) are causing the problem

### Your Code

In [23]:
# Task 3: check shape, isna() on product columns, filter to find problem IDs

# 1
print(f"Original sales shape: {sales.shape}")
print(f"Merged sales with products shape: {sales_w_prod_info.shape}")

Original sales shape: (2581, 8)
Merged sales with products shape: (2581, 11)


In [25]:
# 2
product_cols = ["name", "category", "cost"]
missing = sales_w_prod_info[product_cols].isna().sum()

print(f"Missing values in product columns:\n{missing}")

Missing values in product columns:
name        13
category    13
cost        13
dtype: int64


In [41]:
# 3
missing_info = sales_w_prod_info[
    (sales_w_prod_info["name"].isna()) |
    (sales_w_prod_info["category"].isna()) |
    (sales_w_prod_info["cost"].isna())
]

missing_prod_ids = missing_info["product_id"].unique()
print("product_id values with missing product info:", missing_prod_ids)

product_id values with missing product info: ['201' '199' '200']


## Task 4: Profit by Category

First, calculate profit for each transaction: `profit = revenue - (cost × quantity)`

Then summarize profitability by product category. Your code should return a table that shows both total profit and number of transactions for each category. Sort it by total profit (highest first).

### Your Code

In [42]:
# Task 4: create profit column, then groupby with .agg()
sales_w_prod_info["profit"] = (
    sales_w_prod_info["revenue"]
    - sales_w_prod_info["cost"] * sales_w_prod_info["quantity"]
)

In [43]:
sales_w_prod_info.head()

,transaction_id,timestamp,location,product_id,quantity,revenue,payment_method,loyalty_member,name,category,cost,profit
0,1003,2025-11-13 06:00:00,Campus,108,1.0,2.61,Credit,False,Chai Latte,Tea,1.2,1.41
1,1001,2025-11-13 06:13:00,Campus,104,1.0,2.17,Mobile,False,Americano,Coffee,1.0,1.17
2,1000,2025-11-13 06:44:00,Downtown,103,1.0,5.89,Credit,False,Latte,Coffee,1.9,3.99
3,1002,2025-11-13 06:49:00,Downtown,101,2.0,6.48,Credit,False,Espresso,Coffee,1.2,4.08
4,1016,2025-11-13 07:03:00,Campus,102,1.0,4.43,Credit,False,Cappuccino,Coffee,1.8,2.63


In [64]:
profit_by_cat = sales_w_prod_info.groupby("category").agg({
    "profit": "sum",
    "transaction_id": "count"
})

In [79]:
profit_by_cat_srtd = profit_by_cat.sort_values(
    "profit", 
    ascending=False
)

profit_by_cat_srtd = profit_by_cat_srtd.rename(columns = {
    "profit": "total_profit",
    "transaction_id": "number_of_transactions"
})

In [80]:
profit_by_cat_srtd

,total_profit,number_of_transactions
category,,
Sandwich,4295.03,598
Coffee,2292.26,918
Pastry,1394.47,484
Tea,801.90,568


## Task 5: Profit by Category and Location

Analyze how profitability varies by both category and location. Your code should produce a table that shows the following for every combination of category and location:

- Total profit
- Average profit per transaction
- Number of transactions

Display the results as a flat table (i.e., without a nested/hierarchical index).

### Your Code

In [86]:
# Task 5: groupby multiple columns, .agg() with dict, reset_index()

profit_cat_loc = sales_w_prod_info.groupby(["category", "location"]).agg({
    "profit": "sum",
    "transaction_id": "count"
})

In [87]:
profit_cat_loc["avg_profit_per_transaction"] = (
    profit_cat_loc["profit"] / profit_cat_loc["transaction_id"]
)

In [88]:
profit_cat_loc = profit_cat_loc.reset_index()

profit_cat_loc = profit_cat_loc.rename(columns = {
    "profit": "total_profit",
    "transaction_id": "number_of_transactions"
})

In [89]:
profit_cat_loc

,category,location,total_profit,number_of_transactions,avg_profit_per_transaction
0,Coffee,Campus,905.00,453,1.997792
1,Coffee,Downtown,1196.22,387,3.091008
2,Coffee,Suburb,191.04,78,2.449231
3,Pastry,Campus,526.27,213,2.470751
4,Pastry,Downtown,563.76,164,3.437561
5,Pastry,Suburb,304.44,107,2.845234
6,Sandwich,Campus,762.54,140,5.446714
7,Sandwich,Downtown,2446.76,299,8.183144
8,Sandwich,Suburb,1085.73,159,6.828491
9,Tea,Campus,186.65,168,1.111012


## Task 6: Visualize Category Profitability

Create a bar chart showing total profit by product category. Use seaborn and ensure that the result:

- Has clearly labeled categories
- Includes a descriptive title
- Makes it clear which category is most/least profitable

### Your Code

In [ ]:
# Task 6: use sns.barplot() with aggregated data


## Interpretation

Concisely answer the following questions based on your analysis.


1. What might go wrong if you did not verify the merge in Task 3 before proceeding with the analysis?
2. In Tasks 4 and 5, you aggregated data at different levels of detail (by category, then by category and location). How did this change what you could learn from the data?
3. Reflect on the patterns you discovered in your analysis. What valuable insights did your work generate?
4. The coffee shop owner asks: "Based on your analysis, what's the single most important thing I should do to improve profitability?" Provide a specific, actionable recommendation. Justify your recommendation with evidence from your analysis (include specific numbers from your results). Also, suggest additional analysis that you feel might be worthwhile.

### Your Answers

*Interpretation responses here*

## Bonus: Time-Based Analysis (Optional)

For extra credit, investigate a time-based question of your choosing. Examples of questions you could explore include:

- Do sales patterns differ between weekdays and weekends?
- Which hours are busiest at each location?
- Is there a day of the week that stands out for profitability?

For full credit you must:

- State your question clearly
- Show your analysis code
- Summarize what you found in 2-3 sentences

In [ ]:
# Bonus: Your time-based analysis here


### Your Findings

*Bonus findings here (state your question and what you discovered)*

## Submission

Ensure your notebook:

- Has the verification block (Banner ID and code) as the first cell
- Runs completely from top to bottom without errors
- Shows all task outputs clearly
- Includes all interpretation responses
- Is saved and committed to your repository

Follow the submission instructions on Canvas to submit your work.